# Localización de Instalaciones con Capacidad y Asignación Única (CFLP)

---

## Introducción

El **problema de localización de instalaciones con capacidad y asignación única** (CFLP, por sus siglas en inglés) es un problema clásico de optimización combinatoria que busca determinar la ubicación óptima de instalaciones para satisfacer la demanda de un conjunto de clientes, minimizando el coste total de apertura y transporte.

### Enunciado del problema

Dado un conjunto de clientes con demandas específicas y un conjunto de instalaciones con capacidades limitadas, el objetivo es determinar qué instalaciones abrir y cómo asignar los clientes a estas, minimizando el coste total, sujeto a las siguientes restricciones:

- Cada cliente debe ser atendido por una única instalación.
- Las instalaciones tienen una capacidad máxima que no puede ser excedida.
- Solo se puede asignar un cliente a una instalación si esta está abierta.

### Variables de decisión:

- $x_{ij}$: Variable binaria que toma el valor 1 si el cliente $i$ es atendido por la instalación $j$, y 0 en caso contrario.
- $y_{j}$: Variable binaria que toma el valor 1 si la instalación $j$ está abierta, y 0 en caso contrario.

### Parámetros:

- $c_{ij}$: Coste de transporte por asignar el cliente $i$ a la instalación $j$.
- $f_{j}$: Coste fijo por abrir la instalación $j$.
- $d_{i}$: Demanda del cliente $i$.
- $s_{j}$: Capacidad de la instalación $j$.

### Función objetivo:

El objetivo es minimizar el coste total, compuesto por los costes de apertura y transporte:

$$
\text{Minimizar} \quad \sum_{j} f_{j} \cdot y_{j} + \sum_{i} \sum_{j} c_{ij} \cdot x_{ij}
$$

### Restricciones:

1. **Cada cliente debe ser atendido por una única instalación:**

$$
\sum_{j} x_{ij} = 1 \quad \forall i
$$

2. **No se puede asignar un cliente a una instalación que no está abierta:**

$$
 x_{ij} \leq y_{j} \quad \forall i, \forall j
$$

3. **Capacidad de las instalaciones:**

$$
\sum_{i} d_{i} \cdot x_{ij} \leq s_{j} \cdot y_{j} \quad \forall j
$$

4. **Variables binarias:**

$$
 x_{ij}, y_{j} \in \{0, 1\}
$$

---

## Generación de Instancias Aleatorias

Para resolver el problema, se generarán instancias aleatorias que incluyan:

1. **Coordenadas**:
   - Coordenadas aleatorias en un plano bidimensional para representar la ubicación de los clientes e instalaciones.

2. **Costos de Transporte**:
   - Calculados como la distancia euclidiana entre clientes e instalaciones.

3. **Demandas y Capacidades**:
   - Demandas aleatorias para cada cliente.
   - Capacidades aleatorias para cada instalación.

4. **Costos de Apertura**:
   - Valores aleatorios asociados a cada instalación.

---

## Resolución del problema
A continuación, se presenta una implementación del Problema de Localización de Instalaciones con Capacidad y Asignación Única (CFLP) utilizando el paquete JuMP en Julia y el optimizador GLPK. 

In [2]:
using JuMP, GLPK, Random, LinearAlgebra

# Generación de instancias aleatorias
function generate_instance(num_clients, num_facilities)
    Random.seed!(1234)
    coord_clients = rand(0:100, num_clients, 2)
    coord_facilities = rand(0:100, num_facilities, 2)

    transport_costs = [round(sqrt(sum((coord_clients[i, :] .- coord_facilities[j, :]).^2))) for i in 1:num_clients, j in 1:num_facilities]
    opening_costs = rand(50:100, num_facilities)
    demands = rand(5:15, num_clients)
    capacities = rand(20:50, num_facilities)

    return transport_costs, opening_costs, demands, capacities
end

# Resolver el modelo CFLP
function solve_cflp(transport_costs, opening_costs, demands, capacities)
    num_clients = size(transport_costs, 1)
    num_facilities = size(transport_costs, 2)

    model = Model(GLPK.Optimizer)

    # Variables de decisión
    @variable(model, x[1:num_clients, 1:num_facilities], Bin)
    @variable(model, y[1:num_facilities], Bin)

    # Función objetivo
    @objective(model, Min, sum(opening_costs[j] * y[j] for j in 1:num_facilities) +
                          sum(transport_costs[i, j] * x[i, j] for i in 1:num_clients, j in 1:num_facilities))

    # Restricción: cada cliente debe ser asignado a una única instalación
    @constraint(model, [i in 1:num_clients], sum(x[i, j] for j in 1:num_facilities) == 1)

    # Restricción: no asignar clientes a instalaciones cerradas
    @constraint(model, [i in 1:num_clients, j in 1:num_facilities], x[i, j] <= y[j])

    # Restricción: capacidad de las instalaciones
    @constraint(model, [j in 1:num_facilities], sum(demands[i] * x[i, j] for i in 1:num_clients) <= capacities[j] * y[j])

    # Resolver el modelo
    optimize!(model)

    if termination_status(model) == MOI.OPTIMAL
        println("\nSolución óptima encontrada:")
        println("Costo total: ", objective_value(model))
        println("Instalaciones abiertas:")
        for j in 1:num_facilities
            if value(y[j]) > 0.5
                println("- Instalación ", j)
            end
        end
        println("\nAsignaciones:")
        for i in 1:num_clients
            for j in 1:num_facilities
                if value(x[i, j]) > 0.5
                    println("Cliente ", i, " asignado a instalación ", j)
                end
            end
        end
    else
        println("No se encontró una solución óptima.")
    end
end

# Parámetros
num_clients = 10
num_facilities = 5

# Generar datos y resolver el modelo
transport_costs, opening_costs, demands, capacities = generate_instance(num_clients, num_facilities)
solve_cflp(transport_costs, opening_costs, demands, capacities)


Solución óptima encontrada:
Costo total: 458.0
Instalaciones abiertas:
- Instalación 2
- Instalación 3
- Instalación 5

Asignaciones:
Cliente 1 asignado a instalación 3
Cliente 2 asignado a instalación 3
Cliente 3 asignado a instalación 2
Cliente 4 asignado a instalación 2
Cliente 5 asignado a instalación 3
Cliente 6 asignado a instalación 3
Cliente 7 asignado a instalación 5
Cliente 8 asignado a instalación 5
Cliente 9 asignado a instalación 5
Cliente 10 asignado a instalación 5
